In [ ]:
import importlib
import data.VCTK
import model.model
import model.trainer
import torch

importlib.reload(data.VCTK) # prevent cache when making changes
importlib.reload(model.trainer) # prevent cache when making changes
importlib.reload(model.model) # prevent cache when making changes

In [2]:
dataset = data.VCTK.VCTKDataset("VCTK-Corpus-smaller/")

device = torch.device("cuda")
myModel = model.model.SpectrogramModel().to(device)

In [33]:
x = dataset[0][2]
x = x[0].unsqueeze(0)
print(x.shape)
x = x.repeat(50, 1, 1)
print(x.shape)

torch.Size([1, 80, 129])
torch.Size([50, 80, 129])


In [3]:
sum([len(x) for x in list(myModel.parameters())])

129184

In [53]:
torch.cuda.empty_cache()

In [ ]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved

In [22]:
del x

In [23]:
x = dataset[0][2][0,:,:].to(device).repeat(100,1,1)
print(x.shape)

torch.Size([100, 80, 129])


In [20]:
importlib.reload(model.trainer) # prevent cache when making changes
myTrainer = model.trainer.Trainer(myModel, device)

In [27]:
import tqdm.notebook as tqdm
for i, _ in tqdm.tqdm(list(enumerate(range(0,100)))):
    myTrainer.train_step(x, x)

  0%|          | 0/100 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 1.66 GiB (GPU 0; 11.17 GiB total capacity; 8.54 GiB already allocated; 630.81 MiB free; 10.08 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF